## Cyberbully detection using BERT-based models and comparing the perforamnce of general-purpose and domain-specifc models in this classification task


For my project, I am using two language models, RoBERTa and SecureBERT. While RoBERTa is a general-purpose language model that is trained on English text, SecureBERT is a domain-specific language model that is designed specifically for cybersecurity.

For training these models, I use Huggingface framework (transformers package) for defining and processing dataset and creating the Trainer function.

In [ ]:
!pip install datasets
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification
from transformers import RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score
from datasets import Dataset

**Config Class**

In [ ]:
class Config:
    model_name = ['roberta-base', "ehsanaghaei/SecureBERT"]  # 'roberta-base', "ehsanaghaei/SecureBERT"
    learning_rate = 2e-5
    epochs = 5
    batch_size = 18
    num_warmup_steps = 100

**read Dataset**

I am using the cyberbullying dataset available at Huggingface repository

In [ ]:
train_dataset = load_dataset("poleval2019_cyberbullying", 'task01', split="train")
test_dataset = load_dataset("poleval2019_cyberbullying", 'task01', split="test")
len(test_dataset)


1000

In [ ]:
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1'], id=None)}

In [ ]:
def Tokenizer(data):
    return tokenizer(data['text'], padding='max_length', truncation=True)

In [ ]:
def encode_labels(data):
    data['label'] = int(data['label'])
    return data


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1, 'precision': precision}

# **Initializing RoBERTa model and Tokenizer**

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(Config.model_name[0])
model = RobertaForSequenceClassification.from_pretrained(Config.model_name[0])

# --- process the data

# train_dataset = Dataset.from_dict(train_dataset)
# test_dataset = Dataset.from_dict(test_dataset)


train_dataset = train_dataset.map(Tokenizer, batched=True)
test_dataset = test_dataset.map(Tokenizer, batched=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Map:   0%|          | 0/10041 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/10041 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## config Training Args for RoBERTa

In [ ]:
training_args = TrainingArguments(
    output_dir=f"'./{Config.model_name[0]}-results'",
    evaluation_strategy='epoch',
    num_train_epochs=Config.epochs,
    per_device_train_batch_size=Config.batch_size,
    per_device_eval_batch_size=Config.batch_size,
    learning_rate=Config.learning_rate,
    warmup_steps=Config.num_warmup_steps,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True,
    save_strategy='epoch'
)


## Define Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision
1,0.313400,0.432261,0.866000,0.803811,0.749956
2,0.269300,0.515065,0.866000,0.803811,0.749956
3,0.226000,0.395719,0.882000,0.844447,0.876515
4,0.182200,0.400108,0.887000,0.868862,0.870305
5,0.148900,0.478247,0.887000,0.868862,0.870305


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2790, training_loss=0.21928599641314545, metrics={'train_runtime': 4749.3972, 'train_samples_per_second': 10.571, 'train_steps_per_second': 0.587, 'total_flos': 1.32094905343488e+16, 'train_loss': 0.21928599641314545, 'epoch': 5.0})

## Plot Performance Results

In [ ]:
def plot_results():
  train_losses = trainer.get_train_running_loss()
  eval_metrics = trainer.get_eval_metrics()

  # Plot the training and evaluation metrics
  import matplotlib.pyplot as plt

  fig, ax = plt.subplots()
  ax.plot(train_losses, label='Training loss')
  ax.plot(eval_metrics['eval_loss'], label='Evaluation loss')
  ax.plot(eval_metrics['eval_f1'], label='F1 score')
  ax.plot(eval_metrics['eval_accuracy'], label='Accuracy')
  ax.legend()
  plt.show()

plot_results()

AttributeError: ignored

# **SecureBERT**
Process is pretty much the same, I just changed the tokenizer for processing the data and the model for training.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(Config.model_name[1])
model = RobertaForSequenceClassification.from_pretrained(Config.model_name[1])
train_dataset = load_dataset("poleval2019_cyberbullying", 'task01', split="train")
test_dataset = load_dataset("poleval2019_cyberbullying", 'task01', split="test")
train_dataset = train_dataset.map(Tokenizer, batched=True)
test_dataset = test_dataset.map(Tokenizer, batched=True)

Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.we

Map:   0%|          | 0/10041 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/10041 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir=f"'./{Config.model_name[1]}-results'",
    evaluation_strategy='epoch',
    num_train_epochs=Config.epochs,
    per_device_train_batch_size=Config.batch_size,
    per_device_eval_batch_size=Config.batch_size,
    learning_rate=Config.learning_rate,
    warmup_steps=Config.num_warmup_steps,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    load_best_model_at_end=True,
    save_strategy='epoch'
)




In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision
1,0.333800,0.375967,0.866000,0.803811,0.749956
2,0.265700,0.373333,0.866000,0.803811,0.749956


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1116, training_loss=0.2928761718093708, metrics={'train_runtime': 1876.5795, 'train_samples_per_second': 10.701, 'train_steps_per_second': 0.595, 'total_flos': 5283796213739520.0, 'train_loss': 0.2928761718093708, 'epoch': 2.0})

In [ ]:
plot_results()